In [1]:
import sqlite3
import numpy as np
import string
import random
import os
import datetime

In [2]:
def make_connection():
    db_name = "epidemic.db"
    try:
        con = sqlite3.connect(db_name)
        con.row_factory = sqlite3.Row
        cursor = con.cursor()
    except Exception as e:
        print(e)
        print(f"Error connecting to the database")
        return False, False
    return con, cursor

In [3]:
def randomNumber(be, en):
    return np.random.randint(be, en)

In [4]:
def randomString(size, chars = string.ascii_lowercase):
    return ''.join(random.choice(chars) for x in range(size))

In [5]:
def updateStats(values, con, cursor, stats):
    """
    diseaseid, ncases, ndeaths, spreadprob 
    """
#     print('heyyy')
    diseaseid = values[1]
    spreadprob = random.random()
    new_values = [diseaseid, 1, values[2], spreadprob]
    
    query = f"""
        SELECT * from Stats
        WHERE diseaseid = {diseaseid};
            """
    try:
        cursor.execute(query)
        print('first query')
    except Exception as e:
        return_val = str(e) + "Error in executing query for updating stats"
    data = cursor.fetchone()
    print('frist query done')
    print(data)
    if not data:
        query = """
                INSERT INTO Stats
                VALUES (?, ?, ?, ?);
                """
        stats[diseaseid] = new_values[1:]
        print('new query\n\n')
    else:
        query = f"""
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + {values[2]}
                WHERE diseaseid = {diseaseid};
                """
        print('Old update\n')
        new_values = False
        stats.update({diseaseid: [stats[diseaseid][0]+1, stats[diseaseid][1] + values[2], spreadprob]})
    print(f"Doing query {query} and values {new_values}")
    print("\n\n\n\n\n\nREAD THIS")
    if new_values:
        print("one")
        cursor.execute(query, new_values)
    else:
        print("two")
        cursor.execute(query)
    try:
        con.commit()
    except Exception as e:
        return_val = str(e) + "Can't execute the query."
    print(f"thththththththht: {return_val}")    
    return return_val
        

In [6]:
def populateDB(values, type = None, stats = None):
    con, cursor = make_connection()
    return_val = "Done"
    
    if type == 'Hospital':
        query = f"""
                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        """
    elif type == 'Diseases':
        query = f"""
                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        """
    elif type == 'Drugs':
        query = f"""
                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        """
    elif type == 'Pharmacy':
        query = f"""
                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        """
    elif type == 'Cases':
        query = f"""
                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        """
        try:
            print('in try')
            cursor.execute(query, values)
            con.commit()
            print('executed')
#             updateStats(values, con, cursor, stats)
            print('stats updated\n')
        except:
            print("comeon")
            con.close()
    print(query)
    try:
        cursor.execute(query, values)
        con.commit()
        print(type + " query committed")
    except Exception as e:
        return_val = str(e)
    finally:
        con.close()
    print(return_val)
    return return_val

In [7]:
def generateLocation(lat, lon, be, en):
    hosp_loc = []
    for i in range(len(lat)):
        for j in range(10):
            ranNum = random.uniform(be, en)
            longi = lon[i] + ranNum
            lati = lat[i] + ranNum
            loc = str(lati) + ' ' + str(longi)
            hosp_loc.append(loc)
        for j in range(10):
            ranNum = random.uniform(be, en)
            longi = lon[i] + ranNum
            lati = lat[i] - ranNum
            loc = str(lati) + ' ' + str(longi)
            hosp_loc.append(loc)
        for j in range(10):
            ranNum = random.uniform(be, en)
            longi = lon[i] - ranNum
            lati = lat[i] + ranNum
            loc = str(lati) + ' ' + str(longi)
            hosp_loc.append(loc)
        for j in range(10):
            ranNum = random.uniform(be, en)
            longi = lon[i] - ranNum
            lati = lat[i] - ranNum
            loc = str(lati) + ' ' + str(longi)
            hosp_loc.append(loc)
    return hosp_loc

In [8]:
dummyLat = [30.553983, 26.619224, 22.463071, 25.712074, 23.957391, 21.116530, 17.927783, 13.145015, 26.146809]
dummyLong = [77.080078, 75.761719, 76.333008, 79.672852, 83.759766, 79.848633, 77.124023, 77.387695, 92.988281]
be = 0.0001
en = 1.5
locs = generateLocation(dummyLat, dummyLong, be, en)

In [9]:
len(locs)

360

# Populating Hospital

In [10]:
be = 10000
en = 99999

hosp_id = []
hosp_userid = []
hosp_name = []
hosp_loc = []

dummyLat = [30.553983, 26.619224, 22.463071, 25.712074, 23.957391, 21.116530, 17.927783, 13.145015, 26.146809]
dummyLong = [77.080078, 75.761719, 76.333008, 79.672852, 83.759766, 79.848633, 77.124023, 77.387695, 92.988281]
be2 = 0.0001
en2 = 1.5

hosp_loc = generateLocation(dummyLat, dummyLong, be2, en2)

for i in range(len(hosp_loc)):
    ID = randomNumber(be, en)
    print(ID)
    Userid = randomNumber(be, en)
    Name = randomString(randomNumber(6, 12))
    
    hosp_id.append(ID)
    hosp_userid.append(Userid)
    hosp_name.append(Name)
    
    a = populateDB(tuple([ID, Name, hosp_loc[i], Userid]), 'Hospital')

95199

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
39324

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
61956

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
54541

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
11806

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
17377

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
23077

                    INSERT INTO Hospital(id, name, location, us


                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
47016

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
20748

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
23478

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
55043

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
68029

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
52301

                    INSERT INTO Hospital(id, name, location, userid)


Hospital query committed
Done
54635

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
86383

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
60227

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
43659

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
80540

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
88788

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
17798

                    INSERT INTO H

Hospital query committed
Done
85783

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
83431

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
28251

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
99875

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
52230

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
30515

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
83371

                    INSERT INTO H

Hospital query committed
Done
56597

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
27096

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
36356

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
98449

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
65298

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
44555

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
58140

                    INSERT INTO H

Hospital query committed
Done
31093

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
98814

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
64085

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
29611

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
77007

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
24481

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
63560

                    INSERT INTO H

Hospital query committed
Done
30993

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
72057

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
82174

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
17224

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
15403

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
16344

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
23105

                    INSERT INTO H

# Populating Diseases

In [11]:
# disease_id = []
# disease_name = []
disease_mos = []
disease_sym = []

disease_name = ['AIDS', 'Maleria', 'Cholera', 'SmallPox', 'Tuberculosis', 'Influenza', 'Dengue', 'Chikungunya', 'Ebola', 'Meningitis']
disease_id = [1,2,3,4,5,6,7,8,9,10]

for i in range(len(disease_id)):
#     id = randomNumber(be, en)
#     name = randomString(randomNumber(6, 10))
    mos = random.choice(['air', 'water', 'food', 'animal'])
    sym = randomString(100)
    
#     disease_id.append(id)
#     disease_name.append(name)
    disease_mos.append(mos)
    disease_sym.append(sym)
    print(i, disease_id[i])
    a = (populateDB(tuple([disease_id[i], disease_name[i], mos, sym]), 'Diseases'))

0 1

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
1 2

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
2 3

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
3 4

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
4 5

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
5 6

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
6 7

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
7 8

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done


# Populating Drugs

In [12]:
drug_id = []
drug_name = []
drug_req = []
# drug_avail =[]
disease_drug = {}

be2 = 100
en2 = 999

for i in range(len(disease_id)):
    curDisease = disease_id[i]
    for j in range(randomNumber(1,4)):
        id = randomNumber(be2, en2)
        name = randomString(randomNumber(6, 12))
        req = randomNumber(6, 15)
        avail = randomNumber(1500, 3000)
        
        drug_id.append(id)
        drug_name.append(name)
        drug_req.append(req)
#         drug_avail.append(avail)
        
        #storing drug ids corresponding to each disease
        if(curDisease in disease_drug):
            disease_drug[curDisease].append(id)
        else:
            disease_drug[curDisease] = [id]
        print(tuple([id, name, req, disease_id[i]]))
        a = populateDB(tuple([id, name, req, disease_id[i]]), 'Drugs')

(961, 'gncovxz', 8, 1)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(450, 'sqvxdne', 9, 1)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(327, 'qpscwrkcpp', 14, 2)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(264, 'vrptgm', 11, 3)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(931, 'odmivtqidb', 12, 4)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(641, 'tofggiv', 12, 4)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(347, 'cosfuh', 10, 5)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(743, 'yigyaqhwhj', 8, 6)

In [13]:
# !cp backup.db epidemic.db

In [14]:
# !ls

# Populating Cases

In [15]:
# id- auto-incremental
case_date = []
case_diseaseid = []
case_death = []
case_location = []

#stats
stats = {}
#for each disease id in stats, we will store ncases, ndeaths and spreadprob

curDate = str(datetime.datetime.now().date())
for i in range(len(disease_id)):
    curDisease = disease_id[i]
    for j in range(randomNumber(50, 100)):
        date = str(randomNumber(2018, 2020)) + '-' + str(randomNumber(1, 12)) + '-' + str(randomNumber(1, 30))
        if(date < curDate):
            death = randomNumber(0, 2)
            location = random.choice(hosp_loc)

            case_date.append(date)
            case_diseaseid.append(curDisease)
            case_death.append(death)
            case_location.append(location)
            a = populateDB(tuple([date, curDisease, death, location]), 'Cases', stats)

in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, locati

Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(dat

Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(dat


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(

Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(dat

Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(dat

Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(dat

executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
   

Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
in try
executed
stats updated


                    INSERT INTO Cases(dat

In [16]:
# for i, idx in enumerate(disease_id):
#     print(idx)

In [17]:
# updating stats
for idx, dId in enumerate(disease_id):
    con, cursor = make_connection()
    return_val = "Done"
    query = f"""
            SELECT * FROM Cases
            WHERE diseaseid = {dId}
            """
    cursor.execute(query)
    con.commit()
    
    rows = cursor.fetchall()
    data = [dict(row) for row in rows]
    con.close()
    
#     print(data)
    ncases = len(data)
    print(ncases)
    ndeaths = randomNumber(0, ncases)
    spreadprob = random.random()
    
    con, cursor = make_connection()
    query = f"""
            INSERT INTO Stats
            VALUES(?, ?, ?, ?)
            """
    values = [dId, ncases, ndeaths, spreadprob]
    cursor.execute(query, values)
    con.commit()
    con.close()

72
54
76
78
86
68
100
72
102
96


# Populating Pharmacy

In [18]:
def generate_num_array(size, be, en):
    
    numArray = " ".join([str(randomNumber(be, en)) for i in range(size)])
    return numArray

In [19]:
# s = generate_num_array(5, 1, 10)
# s

In [20]:
pharm_id = []
pharm_name = []
pharm_loc = []
pharm_drug_id = []
pharm_drug_avail = []

dummyLat = [33.759740, 29.553983, 27.619224, 22.763071, 25.112074, 24.157391, 21.816530, 17.027783, 13.745015, 26.146809]
dummyLong = [76.677269, 77.080078, 75.761719, 76.333008, 79.672852, 83.759766, 79.848633, 78.124023, 77.387695, 92.988281]
beloc = 0.0001
enloc = 1.5

pharm_loc = generateLocation(dummyLat, dummyLong, beloc, enloc)

for i in range(len(pharm_loc)):
    ID = randomNumber(be, en)
    Name = randomString(randomNumber(6, 12))
    ndrugs = randomNumber(be2 * 10, en2 * 10)
    drug_id = generate_num_array(ndrugs, be2, en2)
    drug_avail = generate_num_array(ndrugs, 10, 200)
    
    pharm_id.append(ID)
    pharm_name.append(Name)
    pharm_drug_id.append(drug_id)
    pharm_drug_avail.append(drug_avail)
    
    a = populateDB(tuple([ID, Name, pharm_loc[i], drug_id, drug_avail]), 'Pharmacy')


                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

       


                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

       


                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

       


                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

       

Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
UNIQUE constraint failed: Pharmacy.id

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        



                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

       